# Creating a Multi-Layer Spatial Data Cube with Metadata in Python

## 1. Build the Cube from Rasterio Objects
If you have multiple open rasterio dataset readers (e.g., src_slope and src_geo), convert them into an xarray.Dataset where each variable is a layer.


In [ ]:
from itertools import compress

import rasterio
import rioxarray
import xarray as xr

example = "../data/example.tif"

src_example = rasterio.open(example)
src_slope = src_example
src_geo = src_example

profile = src_example.profile

# Assume src_slope and src_geo are open rasterio datasets
# Step 1: Wrap them into xarray DataArrays using rioxarray
slope_da = rioxarray.open_rasterio(src_slope).squeeze()
geo_da = rioxarray.open_rasterio(src_geo).squeeze()

# Step 2: Combine into a Dataset (this creates your cube structure)
cube = xr.Dataset({
    "slope": slope_da,
    "geophysics": geo_da
})

# Step 3: Add layer-specific metadata
cube.slope.attrs.update({"description": "Terrain slope", "units": "degrees"})
cube.geophysics.attrs.update({"description": "Magnetic field", "units": "nT"})


## 2. Vertically Stack and Save
To save as a multiband GeoTIFF, you must transform the "columns" (variables) into a "vertical stack" (bands). rioxarray uses the variable names as band descriptions in the resulting TIFF.

In [ ]:
# Stack variables into a single 3D DataArray [band, y, x]
stacked_cube = cube.to_array(dim="band")

# Set band coordinates to ensure names are saved in the TIFF header
stacked_cube = stacked_cube.assign_coords(band=["slope", "geophysics"])

# Save as multiband GeoTIFF
stacked_cube.rio.to_raster("final_cube_2025.tif")


In [ ]:
# Low-level alternative with rasterio
profile.update(
    count=stacked_cube.shape[0],
    compress="LZW",
)

with rasterio.open("named_layers.tif", "w", **profile) as dst:
    dst.write(stacked_cube)
    dst.descriptions = ("slope", "magnetic_field")


## 3. Loading with Names Intact
To reload the file and restore the named variables immediately, use the band_as_variable=True parameter.

In [ ]:
# Reload the cube
# Each band name (slope, geophysics) is restored as a Dataset variable
reloaded_cube = rioxarray.open_rasterio("final_cube_2025.tif", band_as_variable=True)

# Access by name and verify CRS/metadata
print(reloaded_cube.slope.attrs)
print(reloaded_cube.rio.crs)


# Why this approach?
- Rasterio Integration: rioxarray.open_rasterio() can take a file path or an already open rasterio.DatasetReader object.
- Verticality: Converting to a DataArray with a band dimension ensures the data is stored in the standard stacked format expected by geospatial software.
- Metadata Retention: Setting long_name or band coordinates ensures that metadata persists through the save/load process.